# TRACECLEANUP

#### Dependencies:
- pandas 0.21.0+
- pandas dataframe as input

#### Notes:
- There are overlapping functions between the cleaning_routine_pre/cleaning_routine_post functions. This is so that users can easily customize the TRACECLEANUP to suit their research needs. Additionally, there exist 'misc()' functions which is are designed to make it easy to add aditional constraints on filter paramaters.
- Due to the structure of the algorithm, it's unessary to select solely pre or post routines depending on your data type. If the input has only dates > 20120206, then TRACECLEANUP will append filtered 'post' data to an empty 'pre' list with no additional computational overhead. 


#### TRACECLEANUP eliminated 29.16% of transactions when tested on Ford transactions.
> dates: 20020701 - 20170630


> transactions: 4,426,827 

In [ ]:
def TRACECELANUP(dataframe):
    
    def cleaning_routine_pre(transactions):
        
        def canc_pre(df):
            canc = df[df['trc_st'] == 'C']['orig_msg_seq_nb']
            match = df[df['msg_seq_nb'].isin(canc)].index
            return list(canc.index) + list(match)

        def reversal_pre(df):
            labels = ['bond_sym_id','trd_exctn_dt','trd_exctn_tm','rptd_pr',\
                  'entrd_vol_qt','yld_pt','rpt_side_cd','cntra_mp_id','trc_st']

            revs = df[df['asof_cd'] == 'R']
            X = df[labels]
            rev = revs[labels]
            return pd.unique(pd.merge(X,rev, how='inner',on=labels,right_index=True,validate='many_to_one').index)

        def agency_transactions_pre(df):
            agency = df[(df['buy_cpcty_cd'] == 'A') | (df['buy_cpcty_cd'] == 'A')]
            return agency[(agency['cmsn_trd'] == 'N') & (agency['cntra_mp_id'] == 'C')].index
        
        def inter_dealer_pre(df):
            interD = df[(df['cntra_mp_id'] == 'D') & (df['rpt_side_cd'] == 'B')].index
            return interD
        
        def misc_pre(df):
            selected = df[(df['trdg_mkt_cd'] == 'P1') | (df['spcl_trd_fl'] == 'Y')].index
            # user_selected = df[(df['column'] == X) | & (df['column'] == X) etc]
            return selected

        dates = sorted(pd.unique(transactions['trd_exctn_dt']))
        cancelations = []
        for date in (dates):
            day_i = transactions[(transactions['trd_exctn_dt'] == date)]
            cancelations.append(canc_pre(day_i))

        step_one = transactions.drop([i for lis in cancelations for i in lis])
        step_two = step_one.drop(reversal_pre(step_one))
        step_three = step_two.drop(agency_transactions_pre(step_two))
        step_four = step_three.drop(inter_dealer_pre(step_three))
        step_five = step_four.drop(misc_pre(step_four))
        return step_five
         

    def cleaning_routine_post(transactions):
    
        def canc_corr_post(df): 
            cancels = df[df['trc_st'] == 'X']['msg_seq_nb']
            y = df[~df['msg_seq_nb'].isin(cancels) & ~df['trc_st'].isin(['X'])]
            corrections = y[y['trc_st'] == 'C']['msg_seq_nb']
            return y[~y['msg_seq_nb'].isin(corrections) & ~y['trc_st'].isin(['C'])].index

        def reversal_post(df):
            X = df
            revs = X[(X['asof_cd'] == 'R') & (X['trc_st'] == 'Y')]
            indeX = X.index
            indeR = revs.index

            drops = []
            for i in indeR:
                R = revs.loc[i]
                lookback = X[(X['trd_exctn_dt'] == R['trd_exctn_dt']) & (X['trd_exctn_tm'] == R['trd_exctn_tm']) & 
                             (X['msg_seq_nb'] == R['orig_msg_seq_nb']) & (X['rptd_pr'] == R['rptd_pr']) & 
                             (X['entrd_vol_qt'] == R['entrd_vol_qt'])  & (X['trc_st'] == 'T') &
                            (R['trd_rpt_dt'] - X['trd_exctn_dt'] > 20)].index.values
                drops.append(lookback)

            matched = [index for lis in drops for index in lis]
            return matched + list(indeR)

        def agency_transactions_post(df):
            agency = df[(df['buy_cpcty_cd'] == 'A') | (df['buy_cpcty_cd'] == 'A')]
            return agency[(agency['cmsn_trd'] == 'N') & (agency['cntra_mp_id'] == 'C')].index
        
        def inter_dealer_post(df):
            interD = df[(df['cntra_mp_id'] == 'D') & (df['rpt_side_cd'] == 'B')].index
            return interD
        
        def misc_post(df):
            selected = df[(df['trdg_mkt_cd'] == 'P1') | (df['spcl_trd_fl'] == 'Y')].index
            # user_selected = df[(df['column'] == X) | & (df['column'] == X) etc]
            return selected
            
        dates = sorted(pd.unique(transactions['trd_exctn_dt']))
        not_error = []
        for i in range(len(dates)):
            day_i = transactions[(transactions['trd_exctn_dt'] == dates[i])]
            not_error.append(canc_corr_post(day_i).values)

        step_one =  transactions.loc[[index for lis in not_error for index in lis]]
        step_two = step_one.drop(reversal_post(step_one))
        step_three = step_two.drop(agency_transactions_post(step_two))
        step_four = step_three.drop(inter_dealer_post(step_three))
        step_five = step_four.drop(misc_post(step_four))
        return step_five

    pre_data = dataframe[dataframe['trd_exctn_dt'] < 20120206]
    post_data = dataframe[dataframe['trd_exctn_dt'] >= 20120206]
    
    pre = cleaning_routine_pre(pre_data)
    post = cleaning_routine_post(post_data)
    return pre.append(post)